In [1]:
"""
Imports Here
"""
from dataset import trainloader
from dataset import testloader
from dataset import trainshape
from dataset import testshape

/Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 6): Library not loaded: @rpath/libpng16.16.dylib
  Referenced from: /Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Reason: Incompatible library version: image.so requires version 56.0.0 or later, but libpng16.16.dylib provides version 54.0.0'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
from model import VisionTransformer
from model import Attention
from model import TransBlock

In [30]:
TRAIN_LOSS = []
TRAIN_ACC = []

def train(model, dataloader, accloader, lossfunc, optimiser, lr=0.1, momentum=0.9, batch_size=16, nepochs=10):
    device = next(model.parameters()).device # check what device the net parameters are on
    
    """training"""
    for i in range(nepochs): # for each epoch
        epoch_loss = 0
        model.train()
        n_batches = 0
        for (x_batch, y_batch) in dataloader: # for each mini-batch
            time1 = time.time()
            optimiser.zero_grad()
            loss = lossfunc(model.forward(x_batch), y_batch)
            loss.backward()
            optimiser.step()
            epoch_loss += loss
            n_batches += 1
            time2 = time.time()
            print(time2-time1)
            if n_batches > 4:
                break
        epoch_loss /= n_batches
    
        """evaluating"""
        model.eval()
        accuracy = test(model, accloader, batch_size=batch_size)

        """get performance"""
        TRAIN_LOSS.append(epoch_loss)
        TRAIN_ACC.append(accuracy)
        
        if i > 3:
            break

def test(model, dataloader, batch_size=1):
    with torch.no_grad(): # disable automatic gradient computation for efficiency
        device = next(model.parameters()).device
        
        """make predictions"""
        pcls = []
        items = 0
        for x, y in dataloader:
            if items == 0:
                print("shape:", x.shape, y.shape)
            time1=time.time()
            x = x.to(device)
            pcls.append(y.cpu()-torch.max(model(x), 1)[1].cpu())
            items += 1
            time2=time.time()
            print(time2-time1)

        """get accuracy"""
        pcls = torch.cat(pcls) # concat predictions on the mini-batches
        accuracy = pcls.sum().float() / items
        return accuracy

In [4]:
shape = trainshape()

In [5]:
print(shape)

torch.Size([1076, 4800, 256])


In [47]:
batchsize=
N, Np, P = trainshape()
model = VisionTransformer(inputsize=(batchsize, Np, P), nblocks=2)

In [48]:
loader = trainloader(batchsize=batchsize)
accloader = trainaccloader()

In [49]:
import time
import torch
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimiser = optim.AdamW(model.parameters(), lr=1e-4)

In [50]:
#train(model, trainloader1, criterion, optimiser)
test(model, loader, accloader)

# TIMES = []
# LOSS = []
# i = 0
# for (x, y) in trainloader1:
#     time1 = time.time()
#     criterion.zero_grad()
#     out = model(x)
#     loss = criterion(out, y.type(torch.LongTensor))
#     LOSS.append(loss.item())
#     loss.backward()
#     optimiser.step()
#     time2 = time.time()
#     TIMES.append(time2-time1)
#     print(time2-time1)
#print("TIMES:", TIMES)
#print("LOSS:", LOSS)

shape: torch.Size([4, 4800, 256]) torch.Size([4])
1.6988067626953125
1.6274349689483643
2.037522792816162
2.5007779598236084
2.4892430305480957
2.2384021282196045
2.03725004196167
2.2166709899902344
2.241147994995117
2.036005973815918
2.048389196395874
1.8653678894042969
1.885908842086792
1.8958747386932373
1.8863003253936768
1.9965801239013672
2.178816795349121
2.106549024581909
1.9829299449920654
1.8715488910675049
1.8145999908447266
1.8260700702667236
1.8493447303771973
1.8404717445373535
1.8289730548858643
1.8821001052856445
1.874634027481079
1.8917570114135742
1.854382038116455
1.8868191242218018
2.2410099506378174
2.0835301876068115
2.0032730102539062
1.9688708782196045
1.9962589740753174
1.9557182788848877
2.1488640308380127
2.0861761569976807
2.0471909046173096
2.059058904647827
2.0342600345611572
2.2233188152313232
2.085871696472168
2.0418989658355713
2.0245487689971924
1.991652011871338
2.025761127471924
2.067377805709839


KeyboardInterrupt: 

In [ ]:
print(sum(param.numel() for param in model.parameters()))

In [ ]:
for param in model.parameters():
    print(param.numel())

In [ ]:
print(TRAIN_LOSS)